In [1]:
from ola_models import * 
from ola_RNN import * 

import os, time, copy, math, re, json, pickle, random
import numpy as np
import pandas as pd

import torch, torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
%matplotlib inline 
import seaborn as sns

from functools import partial 

cuda_available = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda_available else "cpu")
print(f'''using device {device}''')

path = !pwd
path = path[0]
print(path)

using device cuda:0
/home/r2/Documents/RNNexp


In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [4]:
class LSTM(nn.Module):
    def __init__(self, x_sz, h_sz, c_sz):
        super(LSTM,self).__init__()
        self.x_sz = x_sz
        self.h_sz = h_sz
        self.c_sz = c_sz

        self.x_lay = nn.Linear(self.x_sz,4*self.h_sz)                
        self.h_lay = nn.Linear(self.h_sz,4*self.h_sz)
        self.c_lay = nn.Linear(self.c_sz,2*self.h_sz)
        
        self.ig_sig = nn.Sigmoid()
        self.fg_sig = nn.Sigmoid()
        self.og_sig = nn.Sigmoid()
        self.gg_tan = nn.Tanh()
        
        self.softmax = nn.LogSoftmax(dim=1)   
            
    def forward(self,input,hidd,cell):        
        x = self.x_lay(input)   
        h = self.h_lay(hidd)           

        x_i,x_f,x_o,x_g = x.chunk(4,1)
        h_i,h_f,h_o,h_g = h.chunk(4,1)        
        
        forget_gate = self.fg_sig(x_f + h_f)        
        input_gate  = self.ig_sig(x_i + h_i)        
        gate_gate   = self.gg_tan(x_g + h_g)         
        output_gate = self.ot_sig(x_o + h_o)
        
        new_cell    = cell * forget_gate + input_gate * gate_gate
        new_hidden  = output_gate * torch.tanh(new_cell) 
                
        return self.softmax(torch.cat(x,new_hidden)), (new_hidden, new_cell)
    
    def batch_forward(self,xb,yb,h_c,loss_fn):        
        self.train()
        if xb[0,0,1].item() == 1: hidden, cell = self.initHidden(xb.shape[0])
        else: hidden,cell = h_c[0],h_c[1]
        loss = 0
        accu = 0
        for char in range(xb.shape[1]):
            x,y           = xb[:,char],yb[:,char]
            x,y,hidden    = unpad(x,y,hidden)
            if x.shape[0] == 0: break
            output,hidden,cell = self.forward(x,hidden,cell1)
            """divide by the bs used for current character"""
            accu += get_accu(output,y)/x.shape[0]    
            loss += loss_fn(output,y)
        return output,(hidden.detach(),cell.detach()),loss/(char+1),accu/(char+1)

    def initHidden(self, bs):
        return (cuda(torch.zeros(bs,self.h_sz)), cuda(torch.zeros(bs,self.c_sz)))

In [5]:
bs  = 30
sql = 1000
lr  = 0.0005 

sched = combine_scheds([0.15, 0.25, 0.2, 0.4], [sched_cos(0.0005, 0.0008), sched_cos(0.0008, 0.0005),sched_lin(0.0005,0.0005),sched_cos(0.0005,0.00005)]) 
data          = pp_trumpdata(path+"/data/trump/", [0.9,0.95], bs)
data.train_dl = TweetDataLoader(data,data.train.tweets,bs,sql,shuffle=True)
data.valid_dl = TweetDataLoader(data,data.valid.tweets,bs,30,shuffle=False)

In [6]:
model  = cuda(LSTM(len(data.decoder), 150, len(data.decoder)))
opt    = optim.RMSprop(model.parameters(), lr)
learn  = Learner(model,  nn.NLLLoss(), opt , data, lr=lr)

In [ ]:
fit_rnn(1,learn,cbs)